In [9]:

orthomosaics_dir = "./data/orthomosaics"
kinyonzo_hippos_tif_path = f"{orthomosaics_dir}/kinyonzo_hippos.tiff"
kinyonzo_hippos_tiles_dir = f"{orthomosaics_dir}/kinyonzo_hippos_tiles"


In [29]:
from osgeo import gdal

gdal.Warp(kinyonzo_hippos_tif_path, kinyonzo_hippos_tif_path, dstSRS="EPSG:4326")


<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x7f3c68eb7ea0> >

In [37]:
%%bash

# gdalinfo ./data/orthomosaics/kinyonzo_hippos.tiff

In [32]:
import gdal2tiles


def create_map_tiles(tif_path, tiles_dir, min_zoom=2, max_zoom=16):

    print(f'generating tiles from {tif_path} to {tiles_dir}/')

    options = {
        'kml': False,
        'nb_processes': 8,
        'title': 'Smart Carte',
        'zoom': (min_zoom, max_zoom),
    }

    gdal2tiles.generate_tiles(tif_path, tiles_dir, **options)


In [40]:

create_map_tiles(kinyonzo_hippos_tif_path, kinyonzo_hippos_tiles_dir, min_zoom=4, max_zoom=22)


generating tiles from ./data/orthomosaics/kinyonzo_hippos.tiff to ./data/orthomosaics/kinyonzo_hippos_tiles/
Generating Base Tiles:
0...10...20...30...40...50...60...70...80...90...100
Generating Overview Tiles:
0...10...20...30...40...50...60...70...80...90...100


In [41]:
import os

from common.aws import s3 as s3_utils
from common.constants import DATA_CDN_BASE_URL, S3_DATA_BUCKET


def save_tiles_to_s3(tiles_dir, name, bucket=S3_DATA_BUCKET):

    dir_name = tiles_dir.split('/')[-1]
    object_base = f'orthomosaics/{name}'
    
    print(f'uploading {tiles_dir} to s3://{bucket}/{object_base}')
    
    for root, dirs, files in os.walk(tiles_dir):
        for file in files:
            file_path = os.path.join(root, file)
            sub_path = file_path.replace(tiles_dir, '')
            object_key = f'{object_base}{sub_path}'
            s3_utils.put_item(file_path, bucket, object_key)

    return object_base


In [43]:

save_tiles_to_s3(kinyonzo_hippos_tiles_dir, "kinyonzo_hippos_tiles")


uploading ./data/orthomosaics/kinyonzo_hippos_tiles to s3://smartcarte-data/orthomosaics/kinyonzo_hippos_tiles


'orthomosaics/kinyonzo_hippos_tiles'

In [ ]:
print('done')

In [44]:
import os
import shutil

if os.path.exists(kinyonzo_hippos_tiles_dir):
    shutil.rmtree(kinyonzo_hippos_tiles_dir)